## Import Library

In [ ]:
!conda install gdcm -c conda-forge -y 

In [ ]:
! conda install -c conda-forge gdcm -y;

In [ ]:
import numpy as np
import pandas as pd
import os 
from os import listdir
import matplotlib.pyplot as plt

# dicom processing library
import pydicom
from pydicom.pixel_data_handlers.util import apply_voi_lut
#from fastai.imports import *
from fastai.medical.imaging import *

from skimage import exposure
from colorama import Fore, Style
import cv2 

In [ ]:
os.getcwd()

## Read Data

In [ ]:
path = '../input/siim-covid19-detection'
os.listdir(path)

In [ ]:
std_df = pd.read_csv(path+'/train_study_level.csv')
img_df = pd.read_csv(path+'/train_image_level.csv')

print('study : ',std_df.shape)
print('image : ', img_df.shape)

In [ ]:
std_df.head()

In [ ]:
std_df

In [ ]:
img_df.head()

In [ ]:
img_df

## Data EDA

In [ ]:
std_df.info()

In [ ]:
std_df.isnull().sum()

In [ ]:
train2.isnull().sum()

In [ ]:
col_ls = list(train2.columns)
col_ls

In [ ]:
del col_ls[0:3]
del col_ls[-1]

for i in range(len(col_ls)):
    print(std_df[col_ls[i]].value_counts())

In [ ]:
#del col_ls[0]
for i in range(len(col_ls)):
    print(train2[col_ls[i]].value_counts())
    
plt.figure(figsize = (10, 8))

plt.bar(col_ls, train2[col_ls].sum())

plt.show()

In [ ]:
col_ls = list(std_df.columns)
col_ls

In [ ]:
del col_ls[0]

for i in range(len(col_ls)):
    print(std_df[col_ls[i]].value_counts())

In [ ]:
plt.figure(figsize = (10, 8))

plt.bar(col_ls, std_df[col_ls].sum())

plt.show()  ##  Typical Appearance 가장 많이 나타남 

## Image 

In [ ]:
'''
https://www.kaggle.com/trungthanhnguyen0502/eda-vinbigdata-chest-x-ray-abnormalities
'''
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data
        
    
def plot_img(img, size=(7, 7), is_rgb=True, title="", cmap='gray'):
    plt.figure(figsize=size)
    plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()


def plot_imgs(imgs, cols=2, size=7, is_rgb=True, title="", cmap='gray', img_size=(500,500)):
    rows = len(imgs)//cols + 1
    fig = plt.figure(figsize=(cols*size, rows*size))
    for i, img in enumerate(imgs):
        if img_size is not None:
            img = cv2.resize(img, img_size)
        fig.add_subplot(rows, cols, i+1)
        plt.imshow(img, cmap=cmap)
    plt.suptitle(title)
    plt.show()

In [ ]:
normal_path

In [ ]:
std = train2[]
img = dicom2array('../input/siim-covid19-detection/train/ff0879eb20ed/d8a644cc4f93/000c3a3f293f.dcm')
plot_img(img)


In [ ]:
dicom_paths = get_dicom_files(path+'/train')
#img = [dicom2array(path), print(path) for path in dicom_paths[:2]]

for path in dicom_paths[:2]:
    print(path)

In [ ]:
std_fold = train2['StudyInstanceUID']
img_name = train2['image_name']

In [ ]:
train2

In [ ]:
path = '../input/siim-covid19-detection/train'
tmp = []
for i in range(3):
    fold = std_fold[i]
    #print(fold)
    name = img_name[i]
   #print(name)
    normal_path = glob.glob(f'{path}/{fold}/*/{name}.dcm')[0]
    img = dicom2array(path = normal_path)
    plot_img(img)
    
    # ../input/siim-covid19-detection/train/00086460a852/9e8302230c91/65761e66de9f.dcm

In [ ]:
for i in tmp:
    img = dicom2array(tmp[i][0])
    plot_img(img)

In [ ]:
train2

In [ ]:
#train2 = pd.read_csv('covid_train2.csv')

#train2

In [ ]:
name = '6534a837497d'
a = glob.glob(f'{path}/00f9e183938e/*/*')
a = glob.glob(f'{dataset_dir}/train/00f9e183938e/*/{name}.dcm')
print(a[0])

In [ ]:
std_fold[0]

In [ ]:
normal_path

In [ ]:
# preprocess - equalize histogram

img = [exposure.equalize_hist(i) for i in img]
plot_imgs(img)

In [ ]:
## Merge Data
'''
https://www.kaggle.com/yujiariyasu/catch-up-on-positive-samples-plot-submission-csv?scriptVersionId=63394385
'''

std_df['StudyInstanceUID'] = std_df['id'].apply(lambda x: x.replace('_study', ''))
del std_df['id']
train = img_df.merge(std_df, on = 'StudyInstanceUID')
train.head()

In [ ]:
img_df.head()

In [ ]:
img_df.isnull().sum()

In [ ]:
img_df['boxes'][0]

In [ ]:
train_new = train.copy()

train_new.shape


In [ ]:
cond1 = train_new['Negative for Pneumonia'] ==1
cond2 = train_new['Typical Appearance'] == 1
cond3 = train_new['Indeterminate Appearance'] == 1
cond4 = train_new['Atypical Appearance'] == 1

In [ ]:
Typi = train_new[cond2].dropna()
Ind = train_new[cond3].dropna()
Atyp = train_new[cond4].dropna()

In [ ]:
new_train2 = pd.concat([train_new[cond1], Typi, Ind, Atyp])

In [ ]:
new_train2

In [ ]:
train2 = new_train2.reset_index()

In [ ]:
del train2['index']

In [ ]:
## box 값 존재하지 않는 경우 삭제 

train2

In [ ]:
tmp = []
for name in train2['id']:
    tmp.append(name[:-6])

In [ ]:
train2['image_name'] = tmp

train2

In [ ]:
## csv로 저장 
train2.to_csv("covid_train2.csv", mode='w')

In [ ]:
train_new[cond1]

In [ ]:
row = train['StudyInstanceUID'] 
a = train['StudyInstanceUID'] == '00f9e183938e'

In [ ]:
new = train[a]

new
    

In [ ]:
train[train['boxes'].isnull()].sum()

## img_df box area distribution

In [ ]:
img_df['area'] = (img_df['scaled_x_max'] - train_with_size['scaled_x_min']) * (train_with_size['scaled_y_max'] - train_with_size['scaled_y_min'])

In [ ]:
plt.figure(figsize=(12,5))
plt.title('typical only: distribution of box area (max 1)')
ax = sns.distplot(train_with_size[train_with_size['Typical Appearance']==1]['area'])
plt.show()

In [ ]:
dataset_dir = '../input/siim-covid19-detection'

In [ ]:
class_names = ['Typical Appearance', 'Indeterminate Appearance', 'Atypical Appearance']
import glob

In [ ]:
'''
https://www.kaggle.com/yujiariyasu/plot-3positive-classes
'''

label2color = {
    '[1, 0, 0]': [255,0,0], # Typical Appearance / red
    '[0, 1, 0]': [0,255,0], # Indeterminate Appearance / green
    '[0, 0, 1]': [0,0,255], # Atypical Appearance / blue
}

def plot_bboxes_with_label(name, n):
    print('Typical Appearance: ' + Fore.RED + 'Red',Style.RESET_ALL)
    print('Indeterminate Appearance: '  + Fore.GREEN + 'Green',Style.RESET_ALL)
    print('Atypical Appearance: ' + Fore.BLUE + 'Blue',Style.RESET_ALL)
    
    imgs = []
    thickness = 2
    scale = 5
    
    if name == 'Negative for Pneumonia':
        label_name = name
        flag = 0
   # elif name == 'Normal':
    #    label_name = 'Negative for Pneumonia'
     #   flag = 1
    else:
        label_name = name
        flag = 1
    
    for _, row in train[train[label_name]==flag].iloc[:n].iterrows():
        study_id = row['StudyInstanceUID']
        img_path = glob.glob(f'{dataset_dir}/train/{study_id}/*/*')[0]
        img = dicom2array(path=img_path)
        img = cv2.resize(img, None, fx=1/scale, fy=1/scale)
        img = np.stack([img, img, img], axis=-1)

        claz = row[class_names].values
        color = label2color[str(claz.tolist())]

        bboxes = []
        bbox = []
        for i, l in enumerate(row['label'].split(' ')):
            '''
            Add comments for others
            '''
            # ================================================================
            # labels, confidence, x_min, y_min, width, height == 6 or NaN == 1
            # ================================================================
            if (i % 6 == 0) | (i % 6 == 1):
                continue
            bbox.append(float(l)/scale)
            if i % 6 == 5:
                bboxes.append(bbox)
                bbox = []
        for box in bboxes:
            img = cv2.rectangle(
                img,
                (int(box[0]), int(box[1])),
                (int(box[2]), int(box[3])),
                color, thickness
        )
        img = cv2.resize(img, (512,512))
        imgs.append(img)
    imgs = [exposure.equalize_hist(i) for i in imgs]
    plot_imgs(imgs, cmap=None)
    
    del img, imgs, bbox, bboxes

In [ ]:
plot_bboxes_with_label('Atypical Appearance', 6)

In [ ]:
ex_train = train.copy()

for _, col in enumerate(ex_train):
    print(col)

class<br>
* Negative for Pneumonia : 0
* Typical Appearanace : 1
* Indeterminate Appearnace : 2
* Atypical Appearance : 3

In [ ]:
ex_train.loc[0, 'Negative for Pneumonia']

In [ ]:
for i in range(len(ex_train)):
    if ex_train.loc[i, 'Negative for Pneumonia'] == 1:
        ex_train.loc[i, 'class_name'] = 0
        
    elif ex_train.loc[i, 'Typical Appearance'] == 1:
        ex_train.loc[i, 'class_name'] = 1
        
    elif ex_train.loc[i, 'Indeterminate Appearance'] == 1:
        ex_train.loc[i, 'class_name'] = 2
        
    elif ex_train.loc[i, 'Atypical Appearance'] == 1:
        ex_train.loc[i, 'class_name'] = 3
        

In [ ]:
ex_train = ex_train.drop(['Negative for Pneumonia', 'Typical Appearance',
                          'Indeterminate Appearance', 'Atypical Appearance'], axis = 1)

In [ ]:
ex_train.head()

In [ ]:
ex_train2 = ex_train.copy()
ex_train2 = ex_train2.drop(['boxes', 'label'], axis = 1)

ex_train2

## Train & Valid split 

In [ ]:
from sklearn.model_selection import train_test_split 

df_train, df_val = train_test_split(ex_train2, test_size = 0.2, random_state = 2359)

print(len(df_train), len(df_val))

In [ ]:
df_train.head()

## Data Augmentation & Transform



In [ ]:
def dicom2array(path, voi_lut=True, fix_monochrome=True):
    dicom = pydicom.read_file(path)
    # VOI LUT (if available by DICOM device) is used to
    # transform raw DICOM data to "human-friendly" view
    if voi_lut:
        data = apply_voi_lut(dicom.pixel_array, dicom)
    else:
        data = dicom.pixel_array
    # depending on this value, X-ray may look inverted - fix that:
    if fix_monochrome and dicom.PhotometricInterpretation == "MONOCHROME1":
        data = np.amax(data) - data
    data = data - np.min(data)
    data = data / np.max(data)
    data = (data * 255).astype(np.uint8)
    return data

In [ ]:
row = df_train.iloc[0]
# f'{dataset_dir}/train/{study_id}/*/*
img_path = glob.glob(f'{dataset_dir}/train/{row.StudyInstanceUID}/*/*')[0]
img = dicom2array(path=img_path)
image = Image.fromarray(img)

print(type(img))
print(type(image))

In [ ]:
import torch 
from torch.utils.data import Dataset
from PIL import Image 

class Classification(Dataset):  
    def __init__(self, df, mode, transform=None):
        self.df = df.reset_index(drop=True)  # random으로 섞인 데이터의 인덱스를 reset 시켜서 다시 부여한다.
        self.transform = transform

    def __len__(self): # 데이터 크기 리턴  # 필수 요소 
        return self.df.shape[0]  # df 행 개수 == 데이터 개수

    def __getitem__(self, index):  # 데이터 해당 이미지 리턴 # 필수 요소 
        # .dcm -> np.array 
        row = self.df.iloc[index]  # 주어진 index에 대한 데이터 뽑아오기
        # f'{dataset_dir}/train/{study_id}/*/*
        img_path = glob.glob(f'{dataset_dir}/train/{row.StudyInstanceUID}/*/*')[0]
        img = dicom2array(path=img_path)
        image = Image.fromarray(img) # PIL.Image.Image
        #image = Image.open(img) # 파일경로로 부터 이미지를 읽기
        target_label = torch.tensor(self.df.iloc[index].class_name).long()
        
        if self.transform:
            image = self.transform(image) # 이미지에 transform 적용하기

        return image, target_label  # 이미지와 class return하기

In [ ]:
from torchvision import transforms

dataset_train = Classification(df_train, 'train', transform = transforms.ToTensor())

print(f'>> 데이터 셋 전체 크기, 개수, sample개수 : {dataset_train.__len__()}')
print(f'>> i번째 데이터 불러오기 : dataset_train.__getitem__(i)')
print(f'>> 0번째 데이터 불러와지는 형태 : tuple (이미지 픽셀값, target)\n : {dataset_train.__getitem__(0)}')

In [ ]:
from torchvision import transforms

def get_transforms(image_size):

    transforms_train = transforms.Compose([
                                       transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(image_size),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor()])
                                       #transforms.Normalize(c_mean, c_std)])

    transforms_val = transforms.Compose([
                                      transforms.Resize(image_size+30),
                                      transforms.CenterCrop(image_size),
                                      transforms.ToTensor()])
                                      #transforms.Normalize(c_mean, c_std)])
    
    return transforms_train, transforms_val

In [ ]:
# 모델 트랜스폼

transforms_train, transforms_val = get_transforms(400)

In [ ]:
# dataset class 객체 

dataset_train = Classification(df_train, 'train', transform=transforms_train)
dataset_val = Classification(df_val, 'valid', transform = transforms_val)

## Data Loader 

In [ ]:
from torch.utils.data.sampler import RandomSampler
from torch.utils.data import DataLoader

train_loader = torch.utils.data.DataLoader(dataset_train, batch_size=4, sampler=RandomSampler(dataset_train), num_workers=0)
valid_loader = torch.utils.data.DataLoader(dataset_val, batch_size=4, num_workers=0)

In [ ]:
for images, targets in train_loader:
    print(f'[batch, channel, ...] : {images.shape}')
    print(targets.shape)

## model 확인 

In [ ]:
from torchvision import models
from collections import OrderedDict
import torch.nn as nn

model = models.vgg16(pretrained=True)

In [ ]:
model

In [ ]:
# Parameter 동결 
for param in model.parameters():
    param.requires_grad = False
    
# 마지막 layer 수정
classifier = nn.Sequential(OrderedDict([
                            ('fc1', nn.Linear(25088, 1000)),
                            ('relu1', nn.ReLU()),
                            ('fc2', nn.Linear(1000, 256)),
                            ('relu2', nn.ReLU()),
                            ('fc3', nn.Linear(256, 4))
                            ]))

model.features[0] = nn.Conv2d(1, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))

model.classifier = classifier

In [ ]:
model

## Data classification - Training

In [ ]:
import numpy as np
import cv2
import random
import time
import torch.optim as optim

from tqdm import tqdm

In [ ]:
def train_epoch(model, loader, device, criterion, optimizer):

    model.train()   # 모델 train 모드로 바꾸기
    train_loss = []
    bar = tqdm(loader)
    for i, (data, target) in enumerate(bar):

        optimizer.zero_grad()  # 최적화된 모든 변수 초기화
        
        data, target = data.to(device), target.to(device)  # 지정한 device로 데이터 옮기기
        logits = model(data)              # 1. forward pass
        
        loss = criterion(logits, target)  # 2. loss 계산
        loss.backward()                   # 3. backward pass

        optimizer.step()                  # 4. gradient descent(파라미터 업데이트)

        loss_np = loss.detach().cpu().numpy() # loss값 가져오기 위해 gpu에 있던 데이터 모두 cpu로 옮기기
        train_loss.append(loss_np)
        bar.set_description('loss: %.5f' % (loss_np))

    train_loss = np.mean(train_loss)  # 한 epoch당 train loss의 평균 구하기
    return train_loss

In [ ]:
# Validation

def val_epoch(model, loader, device, criterion):

    model.eval()  # 모델 evaluate 모드로 바꾸기
    val_loss = []
    LOGITS = []
    PROBS = []
    TARGETS = []
    with torch.no_grad():
        for (data, target) in tqdm(loader):
            
            data, target = data.to(device), target.to(device) # 지정한 device로 데이터 옮기기
            logits = model(data) # 1. forward pass
            probs = logits.softmax(1)  # 다중분류 -> 각 클래스일 확률을 전체 1로 두고 계산하기

            LOGITS.append(logits.detach().cpu())
            PROBS.append(probs.detach().cpu())
            TARGETS.append(target.detach().cpu())

            loss = criterion(logits, target) # 2. loss 계산
            val_loss.append(loss.detach().cpu().numpy())

    val_loss = np.mean(val_loss)
    LOGITS = torch.cat(LOGITS).numpy()
    PROBS = torch.cat(PROBS).numpy()
    TARGETS = torch.cat(TARGETS).numpy()

    # accuracy : 정확도
    acc = (PROBS.argmax(1) == TARGETS).mean() * 100.
   
    return val_loss, acc

In [ ]:
## 학습

def run(model, init_lr, n_epochs):
    
    # gpu 사용
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 
    
    # model을 지정한 장치로 옮기기
    model = model.to(device)
    
    # loss function 지정
    criterion= nn.CrossEntropyLoss()
    
    # optimizer로 adam 사용
    optimizer = optim.Adam(model.parameters(), lr=init_lr)

    for epoch in range(1, n_epochs + 1):
        print(time.ctime(), f'Epoch {epoch}')

        train_loss = train_epoch(model, train_loader, device, criterion, optimizer) # train
        val_loss, acc = val_epoch(model, valid_loader, device, criterion) # validation
        
        content = time.ctime() + ' ' + f'Epoch {epoch}, lr: {optimizer.param_groups[0]["lr"]:.7f}, train loss: {train_loss:.5f}, valid loss: {(val_loss):.5f}, Acc: {(acc):.4f}.'
        print(content)

    torch.save(model.state_dict(), 'best_model.pth')


In [ ]:
run(model, init_lr=4e-3, n_epochs=10)